# **Using CapsNet for Video Classification**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

## **Load the datasets**

In [ ]:
# Initialize labels
LABELS = set(["Abuse", "Assault", "Fighting", "Normal", "Robbery", "Vandalism"])

# Initialize the list of images
print("Loading images:")
imagePaths = list(paths.list_images(r'C:\Users\Yash Umale\Documents\6th Sem\Open Lab\Python Files\Crime Detection\Datasets'))

data = []
labels = []

# Loop over the image paths
for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]

    if label not in LABELS:
        continue
    
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    
    data.append(image)
    labels.append(label)

np.array(labels)
np.array(data)

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

# Initialize the training data augmentation object
trainAug = ImageDataGenerator(rotation_range=30, 
                              zoom_range=0.15, 
                              width_shift_range=0.2, 
                              height_shift_range=0.2, 
                              shear_range=0.15, 
                              horizontal_flip=True, 
                              fill_mode="nearest")

# Initialize the validation/testing data augmentation object 
valAug = ImageDataGenerator()

# Define the ImageNet mean subtraction (in RGB order) 
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean
n_epochs = 50